# Migrating otters (R/MySQL/RSQLite)

This script takes an existing MySQL schema and migrates it to an SQLite readable *.db database via R. While impracticable for larger databases, it allows our fictive otter colony to move between ledgers.

In [1]:
# Libraries
library(tidyverse)
library(odbc)
library(RSQLite)
library(DBI)

Warning message:
"package 'ggplot2' was built under R version 4.5.2"
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'odbc' was built under R version 4.5.2"
Warning message:
"package 'RSQLite' was built under R version 4.5.2"
Warning message:
"package 'DBI' was built under R version 4.5.2"


In [ ]:
# Paths
sql_pwd_path <- r"(D:\data\sql_toy_pwd.txt)"

otters_sqlite_dump_path <- r"(D:\data\otters_dump.db)"

In [3]:
# Load up the password
sql_pwd <- read_delim(
    file.path(sql_pwd_path),
    delim = "\t",
    col_names = TRUE,
    show_col_types = FALSE
)

In [4]:
# Establish connections
c_msql <- dbConnect(
	drv = odbc(),
	driver = "MySQL ODBC 9.6 Unicode Driver",
	database = "otters", # "mysql"
	server = "localhost",
	uid = sql_pwd[1, "uid"] |> pull(),
	pwd = sql_pwd[1, "pwd"] |> pull(),
	port = 3306
)

c_sqlite <- dbConnect(SQLite(), otters_sqlite_dump_path)

In [5]:
q <- "
SHOW TABLES;
"

otters_tables <- dbGetQuery(c_msql, q)

In [6]:
vector_of_tables <- otters_tables[[1]]

In [7]:
for (i in seq_along(vector_of_tables)) {
    # Load up a MySQL table
    current_table <- dbReadTable(c_msql, vector_of_tables[i])
    # Write a SQLite table
    dbWriteTable(c_sqlite, vector_of_tables[i], current_table, overwrite = TRUE)
    # Remove the current table from memory
    rm(current_table)
    # Collect garbage
    gc()
}

In [8]:
dbDisconnect(c_msql)
dbDisconnect(c_sqlite)